cleaning 완료한 데이터는 news_cleaned의 daumnews 테이블에 저장함

# DB에서 daumnews 데이터를 데이터프레임으로 가져오기

In [1]:
import sqlite3
import pandas as pd
import re

In [2]:
conn = sqlite3.connect('../data/news_raw.db')
cur = conn.cursor()

In [3]:
query = cur.execute('SELECT * FROM daumnews')

In [4]:
cols = [column[0] for column in query.description]

In [5]:
daum = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)

In [6]:
daum.head()

,id,title,date,body,summary,link
0,1,반복된 음식점 단속에 절망한 26살 청년의 '극단적 선택',2017.08.01. 09:00,"""벌금과 단속 걱정하던 아들, 장사 준비하던 현수막으로 목 메 숨져""결혼까지 미루고...",26살 황승우씨는 지난 30일 오전 경기 남양주시 조안면 막국수집 주방에서 목을 메...,https://v.daum.net/v/20170801090037914
1,2,'즉석만남에 술 한잔 했는데'..휴가철 성범죄 표적된 여성들,2017.08.01. 15:59,#1. 20대 여성 A씨는 지난해 7월 서해 한 해수욕장에 놀러갔다가 우연히 30대...,"본격적인 여름 휴가철을 맞아 해수욕장, 계곡과 같은 피서지에 인파가 몰리면서 성범죄...",https://v.daum.net/v/20170801155924346
2,3,"필리핀서 한국인 관광객 숨진채 발견..""카지노서 거액 잃어""",2017.08.01. 18:41,(하노이=연합뉴스) 김문성 특파원 = 필리핀에서 30대 한국인 관광객이 숨진 채 발...,필리핀에서 30대 한국인 관광객이 숨진 채 발견됐다.\n현지 경찰은 이 한국인이 카...,https://v.daum.net/v/20170801184128126
3,4,"""부인이 공관병 상대 갑질"" 논란 박찬주 육군대장 전역지원(종합)",2017.08.01. 18:50,\n(서울=연합뉴스) 김귀근 이영재 기자 = 공관병에 대한 부인의 '갑질' 의혹을 ...,공관병에 대한 부인의 '갑질' 의혹을 받고 있는 박찬주 육군 2작전사령관(대장)이 ...,https://v.daum.net/v/20170801185020317
4,5,화장실에 몰카 설치해 딸 친구 '찰칵' 목사 입건,2017.08.01. 07:45,(청주=연합뉴스) 이승민 기자 = 청주 흥덕경찰서는 화장실에 '몰래카메라'를 설치해...,경찰에 따르면 A씨는 지난달 29일 오후 3시 25분께 흥덕구의 자신의 집 화장실 ...,https://v.daum.net/v/20170801074552385


In [7]:
daum.shape

(109419, 6)

In [8]:
cur.close()
conn.close()

# Data Cleaning

## body

In [213]:
daum_copy = daum.copy()

In [214]:
def cleaning_body(row):
    '''
    1. 언론사명, 기자명 제거
    2. 기자이메일 제거
    3. 일부 특수 문자, 띄어쓰기, 영어, 한글 등을 제외하고 모두 제거 (ex. 한자)
    4. 음성정보 제공용 텍스트 안내문구 제거
    '''
    # 1. 언론사명, 기자명
        # = 이 있는 경우
    result = re.sub(r'[\[\(【][가-힣A-Za-z0-9\s,\(\)/·]+=[가-힣A-Za-z0-9\s,\(\)/·]+[\]\)】]\s?([가-힣A-Za-z\s]+=)*',
                   '', row).strip()
        # = 이 없는 경우 본문의 중요 정보가 사라지지 않도록 ()는 match에서 제외
    result = re.sub(r'[\[【][가-힣A-Za-z0-9\s,\(\)/·:|]+[\]】]\s?([가-힣A-Za-z\s]+=)*',
                   '', result).strip()
    result = re.sub(r'\(베스트 일레븐\)',
                   '', result).strip()
    
    # 2. 기자이메일, 각종 계정 제거
    result = re.sub(r'([A-Za-z가-힣]+\s?기자)?\s?[A-Za-z0-9]+@([A-Za-z0-9.]+\s*[A-Za-z가-힣 ]*)?',
                   '', result).strip()
    
    # 3. 일부 특수문자(.,?!/@$%~％·∼(), 띄어쓰기, ASCII 문자(\x00-\x7F), 한글(가-힣)
    result = re.sub(r'[^ .,?!/@$%~％·∼()\x00-\x7F가-힣]+',
                   '', result).strip()
    
    # 4. 음성정보 제공용 텍스트 안내문구
    result = re.sub(r'<<[가-힣A-Za-z0-9 ]+>>',
                    '', result).strip()
    return result

In [215]:
daum_copy.body

0         "벌금과 단속 걱정하던 아들, 장사 준비하던 현수막으로 목 메 숨져"결혼까지 미루고...
1         #1. 20대 여성 A씨는 지난해 7월 서해 한 해수욕장에 놀러갔다가 우연히 30대...
2         (하노이=연합뉴스) 김문성 특파원 = 필리핀에서 30대 한국인 관광객이 숨진 채 발...
3         \n(서울=연합뉴스) 김귀근 이영재 기자 = 공관병에 대한 부인의 '갑질' 의혹을 ...
4         (청주=연합뉴스) 이승민 기자 = 청주 흥덕경찰서는 화장실에 '몰래카메라'를 설치해...
                                ...                        
109414    \n[부산=스포츠조선 박상경 기자] 롯데 자이언츠 허문회 감독은 전준우의 재치 있는...
109415    \n[스포츠조선 선수민 기자] 이 정도면 새 외국인 타자 에디슨 러셀(키움 히어로즈...
109416    \n[OSEN=한용섭 기자] 세인트루이스 카디널스 구단에 코로나19 확진자가 추가로...
109417    \n[스포티비뉴스=부산, 고봉준 기자] 롯데 자이언츠가 8월 첫 주말을 연승으로 장...
109418    \n[스포탈코리아] 곽힘찬 기자= 웨인 루니(더비 카운티)가 과거 맨체스터 유나이티...
Name: body, Length: 109419, dtype: object

In [216]:
pp = daum_copy.body.apply(cleaning_body)

In [217]:
pp = pp.apply(cleaning_body)

In [218]:
daum_copy.body = pp

body 없는 행 제거

In [219]:
blank_idx = []
for i in range(len(daum_copy)):
    if daum_copy.loc[i, 'body'] == '':
        blank_idx.append(i)

In [220]:
len(blank_idx)

216

In [221]:
print(daum_copy.shape)
daum_copy.drop(index=blank_idx, inplace=True)
print(daum_copy.shape)

(109419, 6)
(109203, 6)


## summary

In [222]:
def cleaning_summary(row):
    '''
    일부 특수 문자, 띄어쓰기, 영어, 한글 등을 제외하고 모두 제거 (ex. 한자)
    '''
    # 일부 특수문자(.,?!/@$%~％·∼(), 띄어쓰기, ASCII 문자(\x00-\x7F), 한글(가-힣)
    result = re.sub(r'[^ .,?!/@$%~％·∼()\x00-\x7F가-힣]+',
                   '', row).strip()
    return result    

In [223]:
pp2 = daum_copy.summary.apply(cleaning_summary)

In [224]:
daum_copy.summary = pp2

## DB에 저장

In [225]:
conn = sqlite3.connect('../data/news_cleaned.db')
cur = conn.cursor()

In [226]:
daum_copy.to_sql('daumnews', conn)

In [227]:
cur.close()
conn.close()

# Tokeninzing